In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

list = os.listdir("Stocks1/")
number_files = len(list)

def archivo_no_vacio(fpath):
    return True if os.path.isfile(fpath) and os.path.getsize(fpath) > 0 else False

from collections import namedtuple


In [2]:
## hacemos un dict con los datos  de cada empresa

Empresa = namedtuple('Empresa', 'nombre industria sector marketcap')

empresas_df =  pd.read_csv("companies.csv", sep=",")
empresas_df["ticker"] = empresas_df["ticker"].str.lower()
# empresas_df

empresas_df = empresas_df[['ticker', 'company name', 'industry','sector', 'market cap']]
lista_nombres = empresas_df['company name'].tolist()
lista_tickers = empresas_df['ticker'].tolist()
lista_marketcap = empresas_df['market cap'].tolist()
lista_industrias = empresas_df['industry'].tolist()
lista_sectores = empresas_df['sector'].tolist()

dict_datos_empresas = dict()
for indice, ticker in enumerate(lista_tickers):

    nombre = lista_nombres[indice]
    industria = lista_industrias[indice]
    marketcap = lista_marketcap[indice]
    sector = lista_sectores[indice]

    dict_datos_empresas[ticker] = Empresa(nombre, industria, sector, marketcap)

# empresas_df


import json

with open("datos_empresas.json", "w") as file:
    dicto = json.dumps(dict_datos_empresas)
    file.write(dicto)

In [3]:
# con filtro fecha y filtro 500 empresas
list_files = os.listdir("Stocks1/")
dict_frames = dict() # dicccionario con todos los data frames que rellenaremos
dict_rendimientos = dict()

# no_sirven = 0
#recorremos todas las acciones en Stocks desde 2010 (Stocks1)
for nombre_archivo in list_files:

    # aplicamos filtro de 500 empresas
#     index = nombre_archivo.find(".us")
#     simple = nombre_archivo[0:index]
#     if simple not in list(empresas_df["Symbol"]):
#         continue

    path = f"Stocks1/{nombre_archivo}"

    # recorremos los archivos no vacios
    if archivo_no_vacio(path):

        df =  pd.read_csv(path, sep=",")
        df = df[['Date', 'Open', 'Close', 'Volume']]
        df['ticker'] = nombre_archivo.replace('.csv', '')
        df['Rendimiento'] = (df['Close'] - df['Open']) / df['Open']

        # quitamos los datos nulos
        df.dropna(subset=['Rendimiento'], inplace=True)

        df['Date']= pd.to_datetime(df['Date'], dayfirst = True)

        # filtramos para obtener los datos de hasta el 31 Dic de 2014
        df = df[(df['Date'] > '2015-01-01 00:00:00')]
        df = df[(df['Date'] < '2015-05-01 00:00:00')]

        # se guarda el nombre sin .txt
        dict_frames[nombre_archivo.split(".csv")[0]] = df
        dict_rendimientos[nombre_archivo.split(".csv")[0]] = df[['Rendimiento']]

In [4]:
# # encontremos la maxima presencia bursatil
# max_presencias = 0
# # hacemos un dict, key = nombre empresa, value = % presencia bursatil
# dict_presencias = dict()

# # calculo de la maxima presencia bursatil, valor que sera nuestra referencia
# #vemos las presencia de todas las acciones y elegimos la mayor
# for nombre, df in dict_frames.items():
#     # calculamos presencia bursatil de esa accion
#     pb = len(df.index)
#     if pb > max_presencias:
#         max_presencias = pb

# # guardamos el porcentaje de capitalizacion de cada empresa en el dict
# for nombre, df in dict_frames.items():
#     # calculamos presencia bursatil de esa accion
#     pb_porcentual = len(df.index)/max_presencias
#     dict_presencias[nombre] = pb_porcentual
## meter dicc presencias estandar archivo json ## 
with open('presencias.json') as json_file:
    dict_presencias = json.load(json_file)
dict_presencias

{'a.us': 1.0,
 'aa.us': 1.0,
 'aaap.us': 0.0,
 'aaba.us': 1.0,
 'aac.us': 0.04852824184566428,
 'aal.us': 0.21081941129673826,
 'aamc.us': 0.3937947494033413,
 'aame.us': 0.9578361177406524,
 'aan.us': 0.9992044550517104,
 'aaoi.us': 0.25218774860779636,
 'aaon.us': 0.9992044550517104,
 'aap.us': 0.9992044550517104,
 'aapl.us': 1.0,
 'aat.us': 0.7899761336515513,
 'aau.us': 0.9992044550517104,
 'aav.us': 0.9992044550517104,
 'aaww.us': 0.9992044550517104,
 'aaxn.us': 0.9992044550517104,
 'ab.us': 0.9992044550517104,
 'abac.us': 0.8870326173428799,
 'abax.us': 0.9992044550517104,
 'abb.us': 0.9992044550517104,
 'abbv.us': 0.39777247414478917,
 'abc.us': 1.0,
 'abcb.us': 0.9992044550517104,
 'abcd.us': 0.979315831344471,
 'abco.us': 0.9992044550517104,
 'abdc.us': 0.1288782816229117,
 'abe.us': 0.9976133651551312,
 'abeo.us': 0.003977724741447892,
 'abeow.us': 0.003977724741447892,
 'abev.us': 1.0,
 'abg.us': 0.9992044550517104,
 'abil.us': 0.0,
 'abio.us': 0.9992044550517104,
 'ablx.us'

In [5]:
# filtramos con aquellas empresas que cumplen con el 99,9% de las presencias
# creamos un nuevo dict para evitar cambios del dict en la iteracion
dict_frames_nuevo = dict_frames.copy()
no_sirven = 0
for accion, df in dict_frames.items():
    # si tiene menos del 99,9% de presencias se elimina
    if dict_presencias[accion] < 0.999:
        del dict_frames_nuevo[accion]


# actualizamos las acciones
dict_frames = dict_frames_nuevo

#imprimimos la cantidad de acciones


# agregamos los datos de las empresas
dict_frames_nuevo = dict_frames.copy()
no_sirven = 0
for accion, df in dict_frames.items():
    if accion.split('.us')[0] not in dict_datos_empresas.keys():
        dict_frames_nuevo.pop(accion)
        no_sirven += 1
        continue
    dict_frames_nuevo[accion]['Nombre'] = dict_datos_empresas[accion.split('.us')[0]].nombre
    dict_frames_nuevo[accion]['Sector'] = dict_datos_empresas[accion.split('.us')[0]].sector
#     df['Market Cap'] = dict_datos_empresas[accion.split('.us')[0]].marketcap
# dict_frames_nuevo['a.us']

# df_prueba = dict_frames_nuevo['a.us'].iloc[0]
# df_prueba2 = dict_frames_nuevo['a.us'].iloc[-1]
# df_prueba
# df_prueba2
industrias_utilizadas = []
dic_rango_open= {}
for item in dict_frames_nuevo:
    if item not in industrias_utilizadas:
        industrias_utilizadas.append(item)
        df_open_inicio = dict_frames_nuevo[item].iloc[0]
        df_open_final = dict_frames_nuevo[item].iloc[-1]
        dic_rango_open[item] = [df_open_inicio['Open'],df_open_final['Open'], df_open_final['Open']-df_open_inicio['Open'] ]
    else:
        continue
# dic_peso_industria= {'Industrials':1,'Real Estate':1,'Consumer Defensive':1,'Energy':1,'Basic Materials':1, 'Communication Services':1, ''}
suma_ganancias = 0
total_acciones = 0
for item in dic_rango_open:
    suma_ganancias = +float(dic_rango_open[item][2])
    total_acciones+=1
promedio = suma_ganancias/total_acciones
print(suma_ganancias)
## buscar si la key esta dentro del diccionario y si esta ponderarla por su peso 
### con pesos ## 
# suma_ganancias2 = 0
# total_acciones2 = 0

## tomar la diferencia de precios en dias extremosy multiplicarla por el peso usando la misma key para el dic de peso
#y dic de precios de open

-7.850000000000001


In [7]:
resultados_acciones ={"apu.us": 0.007499999999999983, "artna.us": 0.024999999999999967, "awk.us": 0.019938933922506133, "cdzi.us": 0.007499999999999983, "cpl.us": 0.02088758773231219, "duk.us": 0.020712910808316422, "ed.us": 0.018012446155815046, "kep.us": 0.007499999999999983, "ngg.us": 0.024999999999999967, "pam.us": 0.007499999999999983, "pcg.us": 0.007499999999999983, "ppl.us": 0.020643817591815435, "so.us": 0.024999999999999967, "sph.us": 0.016734093051900746, "tgs.us": 0.007499999999999983, "xel.us": 0.013070210737333639, "bti.us": 0.02500000000000004, "cl.us": 0.018719788426170014, "core.us": 0.019598221345834842, "dltr.us": 0.007500000000000005, "hsy.us": 0.02500000000000004, "iba.us": 0.012669773085624773, "kr.us": 0.02500000000000002, "lway.us": 0.007500000000000005, "mkc.us": 0.010173176419466572, "pg.us": 0.02500000000000002, "sam.us": 0.01633904072290378, "ths.us": 0.02500000000000002, "tues.us": 0.007500000000000005, "wmt.us": 0.02500000000000004, "bkt.us": 0.02500000000000004, "btz.us": 0.02500000000000004, "cb.us": 0.007500000000000005, "egf.us": 0.02500000000000004, "fdef.us": 0.007500000000000005, "jof.us": 0.009526078086907117, "mcr.us": 0.007500000000000005, "mfg.us": 0.007500000000000005, "mmc.us": 0.017251099815181373, "mqy.us": 0.007500000000000005, "mrln.us": 0.007500000000000005, "mui.us": 0.007500000000000005, "smfg.us": 0.008812630511370056, "swz.us": 0.021214966178268797, "tdf.us": 0.010483630680640321, "tree.us": 0.013121699758917036, "tst.us": 0.007500000000000005, "wiw.us": 0.02500000000000004, "wltw.us": 0.009589894968715217, "bt.us": 0.02500000000000004, "ccoi.us": 0.011299111248407023, "chl.us": 0.007500000000000005, "cht.us": 0.007500000000000005, "cmcsa.us": 0.02500000000000004, "dish.us": 0.019026661691407955, "idt.us": 0.007500000000000005, "lbtya.us": 0.022666865556218156, "lbtyk.us": 0.02500000000000004, "sbac.us": 0.009507361503966739, "tkc.us": 0.02500000000000004, "tu.us": 0.02500000000000004, "utsi.us": 0.007500000000000005, "vod.us": 0.02500000000000004, "vz.us": 0.007500000000000005, "parametros": [0.03, 0.1, 0.001, 0.05, 0.25, 0.001]}


In [10]:
import json
with open("Resultados_acciones6.json") as json_file:
    resultados_acciones = json.load(json_file)
resultados_acciones

{'apu.us': 0.009055624983162307,
 'artna.us': 0.010901431818150807,
 'awk.us': 0.011942847704988975,
 'bip.us': 0.007784359021041255,
 'cpl.us': 0.004999999999999989,
 'duk.us': 0.01861405403643265,
 'ed.us': 0.02092089805642076,
 'kep.us': 0.004999999999999989,
 'nee.us': 0.010751758035044759,
 'ngg.us': 0.053681927409332024,
 'nwn.us': 0.004999999999999989,
 'pam.us': 0.004999999999999989,
 'ppl.us': 0.0132924952109907,
 'so.us': 0.04046152022630005,
 'sph.us': 0.01607942114923958,
 'sre.us': 0.011090191178237192,
 'utl.us': 0.005423471170658418,
 'bti.us': 0.01565953394305972,
 'cag.us': 0.005000000000000008,
 'casy.us': 0.005023901824114898,
 'ccu.us': 0.005000000000000008,
 'chd.us': 0.005000000000000008,
 'cl.us': 0.007227339804931753,
 'clx.us': 0.018497775144139382,
 'coke.us': 0.005000000000000008,
 'cpb.us': 0.005000000000000004,
 'dg.us': 0.005000000000000004,
 'flo.us': 0.005073397083293129,
 'hsy.us': 0.007899764214751932,
 'iba.us': 0.009303436196521339,
 'k.us': 0.009246

###### import json
with open("Resultados_acciones.json") as json_file:
    resultados_acciones = json.load(json_file)
resultados_acciones

In [11]:


sum_ganancias2 = 0
total_acciones2 = 0
for item in resultados_acciones:
    
    if item != "parametros":
        sum_ganancias2 = +float(dic_rango_open[item][2]*resultados_acciones[item])
        total_acciones2+=1

promedio = sum_ganancias2/total_acciones2

print(resultados_acciones["parametros"])
print(promedio)
print(sum_ganancias2)


[0.02, 0.3, 0.0001779465, 0.05, 0.25, 0.0001779465]
0.0008256889120525529
0.06853217970036189


In [12]:
# hacemos el calculo de matriz de covarianza por industria con lo que tenemos por mientras
# para los datos iniciales
Empresa = namedtuple('Empresa', 'nombre industria sector marketcap')
originales_df =  pd.read_csv("companies.csv", sep=",")
originales_df["ticker"] = originales_df["ticker"].str.lower()
originales_df = originales_df[['ticker', 'company name', 'industry','sector', 'market cap']]
lista_tickers_original = originales_df['ticker'].tolist()
lista_sectores_original = originales_df['sector'].tolist()


l_sector= []
l_rend=[]
# print(df)
# print(df["Sector"])
#dict que guardara las matrices de covarianza por sector

dict_matrices = dict()
for sector in set(lista_sectores_original):
    df_rendimientos = pd.DataFrame()  # data frame vacio que iremos rellenando
    for accion, df in dict_frames_nuevo.items():
        if sector != df['Sector'].iloc[0]:
            continue
        else:
            df_rendimientos[accion] = df['Rendimiento']

    df_mean = df_rendimientos.mean(axis=1)
    list_df = df_mean.tolist()
    l_rend.append(list_df)
    l_sector.append(sector)

dic_r_diario_x_industria= {}
cont= 0
# for i in l_sector:
#     dic_r_diario_x_industria[i]:l_rend[cont]
#     cont+=1
# print(dic_r_diario_x_industria)
# print(l_rend)
dff = pd.DataFrame()
for i in l_sector:
    print(cont)
    print(l_rend[cont])
    dff[i]=l_rend[cont]
    cont+=1
# print(dff)
dff.dropna(axis='columns')

0
[]
1
[-0.005282584907282296, -0.012545564160597229, -0.012546154218465102, 0.0037334686651886743, 0.00551264556478207, -0.003687751924977703, 0.001843302353731011, 0.00207986253977503, 0.003996582894395751, -0.007009667770000704, 0.009478762809739697, 0.0038143614759282595, 0.0028575588710787847, 0.007640143542365431, -0.0032759117160695247, -0.00030379763755724703, -0.00039008513309761954, -0.015145695941249385, -0.0033403598986068337, -0.005460095441661054, 0.008703583878091968, 0.013668832333914662, 0.0017444928358169667, 0.006686834071706317, -0.006781406173230327, 0.0027368691926979304, 0.0017933616430780923, 0.0025910186406558613, 0.001979544024200012, 0.005754013061360511, 0.0010936092396855435, -0.0004039378690523706, -0.005166637046119725, 0.00299764261084863, -0.003853793483744807, 0.002358032351266695, 0.0056487543940122635, -0.0026107479757264468, -0.00035464904345158463, 0.0004408437829382526, 0.004320142427394322, -0.00046811394286768414, 0.0024344562525484385, -0.00890

,Communication Services,Healthcare,Financial Services,Technology,Industrials,Consumer Defensive,Energy,Basic Materials,Utilities,Real Estate,Consumer Cyclical
0,-0.005283,0.000009,-0.007487,-0.008123,-0.008580,-0.004108,-0.031737,-0.006692,-0.008040,0.006905,-0.012050
1,-0.012546,-0.018119,-0.009167,-0.017890,-0.014074,-0.010449,-0.020196,-0.000958,-0.010243,0.007506,-0.007991
2,-0.012546,0.006786,0.000018,-0.000691,-0.017886,0.006134,-0.019266,-0.000725,0.004802,-0.002915,-0.016207
3,0.003733,0.004818,0.003794,0.007554,-0.004773,0.010290,0.011271,0.004736,0.006630,0.008004,0.005062
4,0.005513,-0.000303,-0.005867,-0.001920,0.008845,-0.004950,-0.002017,-0.001894,-0.005944,0.001649,0.008044
...,...,...,...,...,...,...,...,...,...,...,...
77,0.000300,-0.028207,-0.004443,-0.006909,-0.001084,-0.005982,-0.007082,-0.001701,-0.004359,0.000952,0.000994
78,-0.004487,-0.007194,0.004730,0.004116,-0.005828,0.003548,0.005156,0.008197,0.003510,-0.003513,-0.007647
79,0.008396,-0.006994,0.001386,-0.000545,0.006291,-0.009107,0.019553,0.001106,0.000213,-0.000106,-0.000470
80,-0.006779,-0.022093,-0.005973,-0.013182,-0.003830,-0.012070,0.008031,-0.004200,-0.009535,-0.009139,-0.007543
